In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [ ]:
sales = pd.read_excel("all_data.xlsx")

In [ ]:
sales.head()

In [ ]:
grade_codes = {
    1096: 4,
    2177: 3,
    2189: 4,
    3328: 2,
    3349: 3,
    3889: 0,
    4388: 1,
    4816: 1,
    5136: 3
}

In [ ]:
sales['grade_code'] = sales['magaza_no'].map(grade_codes)

In [ ]:
sales["tarih"] = pd.to_datetime(sales["tarih"])

In [ ]:
sales.head()

In [ ]:
filtered_items = [8011001, 8011003, 8010903, 8011740, 8011002, 8017442, 8011605, 8010810]
filtered_items.sort()

In [ ]:
selected_sales = sales[sales["mal_no"].isin(filtered_items)]

In [ ]:
selected_sales.info()

In [ ]:
selected_sales = selected_sales.sort_values(by=['magaza_no', 'mal_no', 'tarih']).reset_index(drop=True)

In [ ]:
selected_sales.head()

In [ ]:
grouped_sales = selected_sales.groupby(["magaza_no", "mal_no"])["toplam_satis_adet"].apply(list).reset_index()

In [ ]:
grouped_sales.head(20)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats

# DataFrame'in yüklendiğini varsayıyoruz
df = grouped_sales  # Senin paylaştığın dataframe adı

# Her bir magaza_no, mal_no çifti için analiz yapalım
for index, row in df.iterrows():
    sales_data = row['toplam_satis_adet']
    magaza = row['magaza_no']
    mal = row['mal_no']
    
    # Histogram çizimi
    plt.figure(figsize=(4, 3))
    sns.histplot(sales_data, kde=True, bins=20)
    plt.title(f'Sales Distribution - Store {magaza}, Product {mal}')
    plt.xlabel('Sales Count')
    plt.ylabel('Frequency')
    plt.show()
    
    # Dağılım belirleme
    distributions = ['norm', 'expon', 'gamma', 'lognorm', 'weibull_min']  # poisson kaldırıldı
    best_fit = {}
    
    for dist_name in distributions:
        dist = getattr(stats, dist_name)
        params = dist.fit(sales_data)
        
        # KS Testi (Kolmogorov-Smirnov) ile uygunluğu kontrol et
        ks_stat, p_value = stats.kstest(sales_data, lambda x: dist.cdf(x, *params))
        best_fit[dist_name] = p_value
    
    # En iyi dağılımı belirleme
    best_dist = max(best_fit, key=best_fit.get)
    print(f'Magaza {magaza}, Mal {mal} -> En iyi dağılım: {best_dist} (p={best_fit[best_dist]:.4f})')
